In [41]:
from pxr import Usd,UsdGeom,UsdShade,Sdf,Gf,Kind

In [66]:
stage = Usd.Stage.CreateInMemory()

In [88]:
# Materialを作る
materialPath = Sdf.Path('/Looks/MyMaterial')
material = UsdShade.Material.Define(stage, materialPath)
# Shaderを作る
shaderPath = Sdf.Path('/Looks/MyMaterial/Shader')
shader = UsdShade.Shader.Define(stage,shaderPath)
# シェーダーのタイプを指定
shader.CreateIdAttr('UsdPreviewSurface')
# シェーダーに値を入力するためのコネクタを作成
shader.CreateInput('diffuseColor',Sdf.ValueTypeNames.Color3f).Set(Gf.Vec3f(1,0,0))

True

![](https://gyazo.com/741674f95e716744d2dbdc49c2750f8d.png)

こういう状態

In [89]:
# upstreamShaderに対してOutputPortを作成して、コネクト
material.CreateSurfaceOutput().ConnectToSource(shader.ConnectableAPI(), "surface")

True

![](https://gyazo.com/19da4239abb3836d8e981bdaa3b7ce27.png)

materialにCureateSurfaceOutput()でコネクタを作り、
ConnectToSourceでシェーダーのsurfaceと接続。

In [98]:
# Input / Output をチェック
print("MaterialOutput")
for i in material.GetOutputs():
    print(i.GetAttr())
print("MaterialInput")
for i in material.GetInputs():
    print(i.GetAttr())
print("ShaderOutput")
for i in shader.GetOutputs():
    print(i.GetAttr())
print("ShaderInput")
for i in shader.GetInputs():
    print(i.GetAttr())

MaterialOutput
Usd.Prim(</Looks/MyMaterial>).GetAttribute('outputs:displacement')
Usd.Prim(</Looks/MyMaterial>).GetAttribute('outputs:surface')
Usd.Prim(</Looks/MyMaterial>).GetAttribute('outputs:volume')
MaterialInput
Usd.Prim(</Looks/MyMaterial>).GetAttribute('inputs:frame:stPrimvarName')
ShaderOutput
Usd.Prim(</Looks/MyMaterial/Shader>).GetAttribute('outputs:surface')
ShaderInput
Usd.Prim(</Looks/MyMaterial/Shader>).GetAttribute('inputs:diffuseColor')
Usd.Prim(</Looks/MyMaterial/Shader>).GetAttribute('inputs:metallic')
Usd.Prim(</Looks/MyMaterial/Shader>).GetAttribute('inputs:roughness')


In [90]:
# アサイン用のMeshを作る
modelRoot = UsdGeom.Xform.Define(stage, "/TexModel")
Usd.ModelAPI(modelRoot).SetKind(Kind.Tokens.component)
billboard = UsdGeom.Mesh.Define(stage, "/TexModel/card")
billboard.CreatePointsAttr([(-430, -145, 0), (430, -145, 0), (430, 145, 0), (-430, 145, 0)])
billboard.CreateFaceVertexCountsAttr([4])
billboard.CreateFaceVertexIndicesAttr([0,1,2,3])
billboard.CreateExtentAttr([(-430, -145, 0), (430, 145, 0)])
# PrimvarでUVを設定
texCoords = billboard.CreatePrimvar("st", 
                                    Sdf.ValueTypeNames.TexCoord2fArray, 
                                    UsdGeom.Tokens.varying)
texCoords.Set([(0, 0), (1, 0), (1,1), (0, 1)])

UsdShade.MaterialBindingAPI(billboard).Bind(material)

True

![](https://gyazo.com/36c38d958b3b79ad786165c65b9270b0.png)

MaterialBindingAPI で、MaterialPrimとリレーションで接続

In [82]:
stReader = UsdShade.Shader.Define(stage, shaderPath.AppendChild('stReader'))
stReader.CreateIdAttr('UsdPrimvarReader_float2')
stInput = material.CreateInput('frame:stPrimvarName', Sdf.ValueTypeNames.Token)
stInput.Set('st')
stReader.CreateInput('varname',Sdf.ValueTypeNames.Token).ConnectToSource(stInput)

True

![](https://gyazo.com/375373f58fc640900f91df72664bfe26.png)

In [80]:
diffuseTextureSampler = UsdShade.Shader.Define(stage,shaderPath.AppendChild('diffuseTexture'))
diffuseTextureSampler.CreateIdAttr('UsdUVTexture')
diffuseTextureSampler.CreateInput('file', Sdf.ValueTypeNames.Asset).Set("D:/test.png")
diffuseTextureSampler.CreateInput("st", Sdf.ValueTypeNames.Float2).ConnectToSource(stReader.ConnectableAPI(), 'result')
diffuseTextureSampler.CreateOutput('rgb', Sdf.ValueTypeNames.Float3)
shader.CreateInput("diffuseColor", Sdf.ValueTypeNames.Color3f).ConnectToSource(diffuseTextureSampler, 'rgb')

True

![](https://gyazo.com/4b7c493e21556d7cb70ce9bb0670595d.png)

stPrimvarName + PrimvarReader を使用して、Meshに作成した primvar st (UV情報) を取得する。  
そして、テクスチャパスの色を取得して、ShaderのdiffuseColorに値をセットする。  

In [99]:
# PrimvarReaderのコネクションをチェック
for i in stReader.GetInputs():
    print(i.GetAttr())
for i in stReader.GetOutputs():
    print(i.GetAttr())

Usd.Prim(</Looks/MyMaterial/Shader/stReader>).GetAttribute('inputs:varname')
Usd.Prim(</Looks/MyMaterial/Shader/stReader>).GetAttribute('outputs:result')


In [100]:
print(stage.ExportToString())

#usda 1.0
(
    doc = """Generated from Composed Stage of root layer 
"""
)

def "Looks"
{
    def Material "MyMaterial"
    {
        token inputs:frame:stPrimvarName = "st"
        token outputs:surface.connect = </Looks/MyMaterial/Shader.outputs:surface>

        def Shader "Shader"
        {
            uniform token info:id = "UsdPreviewSurface"
            color3f inputs:diffuseColor = (1, 0, 0)
            color3f inputs:diffuseColor.connect = </Looks/MyMaterial/Shader/diffuseTexture.outputs:rgb>
            float inputs:metallic = 0
            float inputs:roughness = 0.4
            token outputs:surface

            def Shader "stReader"
            {
                uniform token info:id = "UsdPrimvarReader_float2"
                token inputs:varname.connect = </Looks/MyMaterial.inputs:frame:stPrimvarName>
                float2 outputs:result
            }

            def Shader "diffuseTexture"
            {
                uniform token info:id = "UsdUVTexture"
       